# Imports

In [1]:
from numpy.random import seed
seed(888)

import tensorflow as tf
tf.compat.v1.set_random_seed(404)

In [2]:
import os
import numpy as np

from time import strftime
from PIL import Image

# Constants

In [3]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

LOG_PATH = 'tensorboard_mnist_digit_logs/'
LOG_PATH2 = 'tensorboard_mnist_digit_logs2/'
LOG_PATH3 = 'tensorboard_mnist_digit_logs3/'

NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH*IMAGE_HEIGHT*CHANNELS

TEST_IMAGE = 'MNIST/test_img.png'

# Get the Data

In [4]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)

Wall time: 451 ms


In [5]:
y_train_all.shape

(60000,)

In [6]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

In [7]:
y_test.shape

(10000,)

In [8]:
%%time

x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)

Wall time: 37.3 s


In [9]:
%%time

x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

Wall time: 5.9 s


In [10]:
x_train_all.shape

(60000, 784)

In [11]:
x_test.shape

(10000, 784)

# Explore

In [12]:
x_train_all[0].shape

(784,)

In [13]:
y_train_all[:5]

array([5, 0, 4, 1, 9])

# Data Preprocessing

In [14]:
# Re_scale

x_train_all, x_test = x_train_all / 255.0, x_test / 255.0

#### Convert target values to one-hot encoding

In [15]:
values =  y_train_all[:5]
np.eye(10)[values]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [16]:
np.eye(10)[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [17]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [18]:
y_train_all.shape

(60000, 10)

In [19]:
y_test = np.eye(NR_CLASSES)[y_test]

In [20]:
y_test.shape

(10000, 10)

### Create validation dataset from training Data

In [21]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]

In [22]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [23]:
x_val.shape

(10000, 784)

In [24]:
x_train.shape

(50000, 784)

# Setup Tensorflow Graph

In [25]:
tf.compat.v1.disable_eager_execution()
X = tf.compat.v1.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name='X')
Y = tf.compat.v1.placeholder(tf.float32, shape=[None, NR_CLASSES], name='labels')

### Neural Network Architecture

#### Hyperparameters

In [26]:
nr_epochs = 50
learning_rate = 1e-3

n_hidden1 = 512
n_hidden2 = 64

In [27]:
def setup_layer(input, weight_dim, bias_dim, name):
     with tf.compat.v1.name_scope(name):
        initial_w = tf.compat.v1.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w = tf.compat.v1.Variable(initial_value=initial_w, name='W') 

        initial_b = tf.compat.v1.constant(value=0.0, shape=bias_dim) 
        b = tf.compat.v1.Variable(initial_value=initial_b, name='B')

        layer_in = tf.compat.v1.matmul(input, w) + b
        
        if name=='out':
            layer_out = tf.compat.v1.nn.softmax(layer_in)
        else:
            layer_out = tf.compat.v1.nn.relu(layer_in)
        
        tf.compat.v1.summary.histogram('weights', w)
        tf.compat.v1.summary.histogram('biases', b)
        
        return layer_out

In [28]:
# Model without dropout
# layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1], 
#                       bias_dim=[n_hidden1], name='layer_1')
# layer_2 = setup_layer(layer_1, weight_dim=[n_hidden1, n_hidden2], 
#                       bias_dim=[n_hidden2], name='layer_2')
# output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES], 
#                      bias_dim=[NR_CLASSES], name='out')

# model_name = f'{n_hidden1}-{n_hidden2} LR{learning_rate} E{nr_epochs}'

In [29]:
layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1], 
                      bias_dim=[n_hidden1], name='layer_1')
layer_drop = tf.compat.v1.nn.dropout(layer_1, keep_prob=0.8, name='dropout_layer')

layer_2 = setup_layer(layer_drop, weight_dim=[n_hidden1, n_hidden2], 
                      bias_dim=[n_hidden2], name='layer_2')
output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES], 
                     bias_dim=[NR_CLASSES], name='out')

model_name = f'{n_hidden1}-DO-{n_hidden2} LR{learning_rate} E{nr_epochs}'

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Tensorboard Setup

In [30]:
# Folder for Tensorboard

folder_name = f'{model_name} at {strftime("%H %M")}'
directory = os.path.join(LOG_PATH3, folder_name)

try:
    os.makedirs(name=directory)
except OSError as exception:
    print(exception.strerror)
else:
    print('Successfully created directories!')


Successfully created directories!


# Loss Optimisation and Metrics

#### Defining Loss Function

In [31]:
with tf.compat.v1.name_scope('loss_calc'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=output))

#### Defining Optimizer

In [32]:
with tf.compat.v1.name_scope('optimizer'):
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
    train_step = optimizer.minimize(loss)

#### Accuracy Metric

In [33]:
with tf.compat.v1.name_scope('accuracy_calc'):
    model_prediction = tf.argmax(output, axis=1, name='prediction')
    correct_pred = tf.equal(model_prediction, tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [34]:
with tf.compat.v1.name_scope('perfomance'):
    tf.compat.v1.summary.scalar('accuracy', accuracy) # Creating summary for tensorboard.
    tf.compat.v1.summary.scalar('cost', loss)

#### Check input images on Tensorboard

In [35]:
with tf.compat.v1.name_scope('show_image'):
    x_image = tf.compat.v1.reshape(X, [-1, 28, 28, 1])
    tf.compat.v1.summary.image('image_input', x_image, max_outputs=4)

# Run Session

In [36]:
sess = tf.compat.v1.Session()

### Setup Filewriter and Merge Summaries

In [37]:
# Merging all summaries

merged_summary = tf.compat.v1.summary.merge_all()

# Creating a filewriter that writes files for tensorboard to read.

train_writer = tf.compat.v1.summary.FileWriter(directory + '/train')

# Giving filewriter the location of the calculations to write on the file that will be read by tensorboard.

train_writer.add_graph(sess.graph)

# creating filewriter for the validation data.

validation_writer = tf.compat.v1.summary.FileWriter(directory + '/validation')

# Initialise all the variables

In [38]:
init = tf.compat.v1.global_variables_initializer()
sess.run(init)

# Batching the Data

In [39]:
size_of_batch = 1000

In [40]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0    # This will help us keep track of where one batch ends and the new batch starts.

In [41]:
def next_batch(batch_size, data, labels):
    
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
    
    end = index_in_epoch
    
    return data[start:end], labels[start:end]

### Training loop

In [42]:
%%time

for epoch in range(nr_epochs):
    #============= Training Dataset =============
    for i in range(nr_iterations):
        
        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)
        
        feed_dictionary = {X:batch_x, Y:batch_y} #This is a batch that will be running.
        
        sess.run(train_step, feed_dict=feed_dictionary) # here we are running the training on the batch.
        
    s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary) #Calculating the accuracy of the above batch.
    
    train_writer.add_summary(s, epoch)
    
    print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')
    
    #============= Validation ===============
    
    summary = sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(summary, epoch)

print('Done Training')        

Epoch 0 	| Training Accuracy = 0.847000002861023
Epoch 1 	| Training Accuracy = 0.859000027179718
Epoch 2 	| Training Accuracy = 0.8679999709129333
Epoch 3 	| Training Accuracy = 0.8730000257492065
Epoch 4 	| Training Accuracy = 0.8740000128746033
Epoch 5 	| Training Accuracy = 0.968999981880188
Epoch 6 	| Training Accuracy = 0.9779999852180481
Epoch 7 	| Training Accuracy = 0.9800000190734863
Epoch 8 	| Training Accuracy = 0.9800000190734863
Epoch 9 	| Training Accuracy = 0.9829999804496765
Epoch 10 	| Training Accuracy = 0.9869999885559082
Epoch 11 	| Training Accuracy = 0.9890000224113464
Epoch 12 	| Training Accuracy = 0.9890000224113464
Epoch 13 	| Training Accuracy = 0.9900000095367432
Epoch 14 	| Training Accuracy = 0.9900000095367432
Epoch 15 	| Training Accuracy = 0.9879999756813049
Epoch 16 	| Training Accuracy = 0.9890000224113464
Epoch 17 	| Training Accuracy = 0.9879999756813049
Epoch 18 	| Training Accuracy = 0.9900000095367432
Epoch 19 	| Training Accuracy = 0.9909999966

# Saving the Model

In [43]:
outputs = {'accuracy_calc/prediction': model_prediction}
inputs = {'X': X}
tf.compat.v1.saved_model.simple_save(sess, 'SavedModel', inputs, outputs)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: SavedModel\saved_model.pb


# Make a Prediction

In [44]:
img = Image.open(TEST_IMAGE)
img

In [45]:
# Convert image to grayscale and convert it into a flat array.

bw = img.convert('L') #This converts the image into black and white.

In [46]:
# We need to invert the colors so that the writing is white and the background is black.

img_array = np.invert(bw)

img_array.shape

(28, 28)

In [47]:
# We now have to flatten the array. For this we will use the numpy ravel method.

test_img = img_array.ravel()

test_img.shape

(784,)

In [48]:
prediction = sess.run(fetches=tf.compat.v1.argmax(output, axis=1), feed_dict={X:[test_img]})

In [49]:
print(f'Prediction for our test image is:{prediction}')

Prediction for our test image is:[2]


# Testing and Evaluation

In [50]:
test_accuracy = sess.run(fetches=[accuracy], feed_dict={X:x_test, Y:y_test})

print(f'Test Accuracy is {test_accuracy[0]:.2%}')

Test Accuracy is 97.91%


# Reset for the Next Run

In [51]:
train_writer.close()
validation_writer.close()
sess.close()
tf.compat.v1.reset_default_graph()

# Code for 1st Part of module

In [52]:
#with tf.compat.v1.name_scope('hidden_layer1'):

#    initial_w1 = tf.compat.v1.truncated_normal(shape=[TOTAL_INPUTS, n_hidden1], stddev=0.1, seed=42) #Creating initial weights
#    w1 = tf.compat.v1.Variable(initial_value=initial_w1, name='w1')     #Asigning a variable to hold them. 
#    # Creating initial Bias
#    initial_b1 = tf.compat.v1.constant(value=0.0, shape=[n_hidden1]) 
#    b1 = tf.compat.v1.Variable(initial_value=initial_b1, name='b1')
#    layer1_in = tf.compat.v1.matmul(X, w1) + b1
#    layer1_out = tf.compat.v1.nn.relu(layer1_in)

# Instructor Challenge

### My Solution

In [53]:
#with tf.compat.v1.name_scope('hidden_layer2'):

#    initial_w2 = tf.compat.v1.truncated_normal(shape=[n_hidden1, n_hidden2], stddev=0.1, seed=42)
#    w2 = tf.compat.v1.Variable(initial_value=initial_w2, name='w2')  

#    initial_b2 = tf.compat.v1.constant(value=0.0, shape=[n_hidden2]) 
#    b2 = tf.compat.v1.Variable(initial_value=initial_b2, name='b2')

#    layer2_in = tf.compat.v1.matmul(layer1_out, w2) + b2
#    layer2_out = tf.compat.v1.nn.relu(layer2_in)

In [54]:
#with tf.compat.v1.name_scope('output_layer'):

#    initial_w3 = tf.compat.v1.truncated_normal(shape=[n_hidden2, NR_CLASSES], stddev=0.1, seed=42)
#    w3 = tf.compat.v1.Variable(initial_value=initial_w3, name='w3') 

#    initial_b3 = tf.compat.v1.constant(value=0.0, shape=[NR_CLASSES]) 
#    b3 = tf.compat.v1.Variable(initial_value=initial_b3, name='b3')

#    layer3_in = tf.compat.v1.matmul(layer2_out, w3) + b3
#    output = tf.compat.v1.nn.softmax(layer3_in)

In [55]:
#b3.eval(sess)